In [1]:
# crash-on tutorial

# 1 Postgresql

In [7]:
## import postgreSQL adapter for the Python
import psycopg2
try: 
    conn = psycopg2.connect("host=127.0.0.1 dbname=studentdb user=student password=student")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)
# Use the connection to get a cursor that can be used to execute queries.
cur = conn.cursor()

#use automatic commit so each action is commited without having to call `conn.commit()` after each command. 
conn.set_session(autocommit=True)

# If you run the insert statement code more than once, you will see duplicates of your data. 
# PostgreSQL allows for duplicates.
try: 
    cur.execute("CREATE TABLE IF NOT EXISTS music_library (album_name varchar, artist_name varchar, year int);")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)

try: 
    cur.execute("INSERT INTO music_library (album_name, artist_name, year) \
                 VALUES (%s, %s, %s)", \
                 ("Let It Be", "The Beatles", 1970))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

try: 
    cur.execute("select * from music_library")
except psycopg2.Error as e:
    print(e)
    
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

# Drop the table to avoid duplicates and clean up
try: 
    cur.execute("DROP table music_library")
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e) 
    
#close
cur.close()
conn.close()

('Let It Be', 'The Beatles', 1970)


In [ ]:
# some usages of Postgre
# the following line could not run but focus on the usage

# using EXTRACT to extract the date and JOIN with other tables
%%sql
SELECT f.title, ci.city, EXTRACT(month FROM p.payment_date) as month, sum(p.amount) as revenue
FROM payment p
JOIN rental r    ON ( p.rental_id = r.rental_id )
JOIN inventory i ON ( r.inventory_id = i.inventory_id )
JOIN film f ON ( i.film_id = f.film_id)
JOIN customer c  ON ( p.customer_id = c.customer_id )
JOIN address a ON ( c.address_id = a.address_id )
JOIN city ci ON ( a.city_id = ci.city_id )
group by (f.title, ci.city, month)
order by month, revenue desc
limit 10;

# groupby and order by
%%sql
SELECT sum(p.amount) as revenue, EXTRACT(month FROM p.payment_date) as month
from payment p
group by month
order by revenue desc
limit 10;

# convert date and to interger
# TO_CHAR(payment_date :: DATE, 'yyyyMMDD')::integer
%%sql
INSERT INTO dimDate (date_key, date, year, quarter, month, day, week, is_weekend)
SELECT DISTINCT(TO_CHAR(payment_date :: DATE, 'yyyyMMDD')::integer) AS date_key,
       date(payment_date)                                           AS date,
       EXTRACT(year FROM payment_date)                              AS year,
       EXTRACT(quarter FROM payment_date)                           AS quarter,
       EXTRACT(month FROM payment_date)                             AS month,
       EXTRACT(day FROM payment_date)                               AS day,
       EXTRACT(week FROM payment_date)                              AS week,
       CASE WHEN EXTRACT(ISODOW FROM payment_date) IN (6, 7) THEN true ELSE false END AS is_weekend
FROM payment;

# Slicing: rating is fixed to 'PG-13'
%%time
%%sql
SELECT dimDate.day,dimMovie.rating, dimCustomer.city, sum(sales_amount) as revenue
FROM factSales
JOIN dimMovie     on (dimMovie.movie_key         = factSales.movie_key)
JOIN dimDate     on (dimDate.date_key         = factSales.date_key)
JOIN dimCustomer on (dimCustomer.customer_key = factSales.customer_key)
WHERE dimMovie.rating = 'PG-13'
GROUP by (dimDate.day, dimCustomer.city, dimMovie.rating)
ORDER by revenue desc
LIMIT  20;


# Dicing: Creating a subcube, same dimensionality, less values for 2 or more dimensions
# dimMovie.rating in ('PG-13', 'PG')
%%time
%%sql
SELECT dimDate.day,dimMovie.rating, dimCustomer.city, sum(sales_amount) as revenue
FROM factSales
JOIN dimMovie     on (dimMovie.movie_key         = factSales.movie_key)
JOIN dimDate     on (dimDate.date_key         = factSales.date_key)
JOIN dimCustomer on (dimCustomer.customer_key = factSales.customer_key)
WHERE dimMovie.rating in ('PG-13', 'PG')
AND dimCustomer.city in ('Bellevue', 'Lancaster')
AND dimDate.day in ('1', '15', '30')
GROUP by (dimDate.day, dimCustomer.city, dimMovie.rating)
ORDER by revenue desc
LIMIT  20;

# Roll-up: city is summed as country
%%time
%%sql
SELECT dimDate.day,dimMovie.rating, dimCustomer.country, sum(sales_amount) as revenue
FROM factSales
JOIN dimMovie     on (dimMovie.movie_key         = factSales.movie_key)
JOIN dimDate     on (dimDate.date_key         = factSales.date_key)
JOIN dimCustomer on (dimCustomer.customer_key = factSales.customer_key)
GROUP by (dimDate.day,  dimMovie.rating, dimCustomer.country)
ORDER by revenue desc
LIMIT  20;

# Drill-down: city is broken up to districts
%%time
%%sql
SELECT dimDate.day,dimMovie.rating, dimCustomer.district, sum(sales_amount) as revenue
FROM factSales
JOIN dimMovie     on (dimMovie.movie_key         = factSales.movie_key)
JOIN dimDate     on (dimDate.date_key         = factSales.date_key)
JOIN dimCustomer on (dimCustomer.customer_key = factSales.customer_key)
GROUP by (dimDate.day, dimCustomer.district, dimMovie.rating)
ORDER by revenue desc
LIMIT  20;

# Group by sets:将list里面的都group by一遍， 省去单独groupby 每个组合的麻烦
# 但是结果里会有None
%%time
%%sql
SELECT dimDate.month,dimStore.country,sum(sales_amount) as revenue
FROM factSales
JOIN dimDate  on (dimDate.date_key  = factSales.date_key)
JOIN dimStore on (dimStore.store_key = factSales.store_key)
GROUP by grouping sets ((), dimDate.month,  dimStore.country, (dimDate.month,  dimStore.country));

# CUBE
# Group by CUBE (dim1, dim2, ..) , produces all combinations of different lenghts in one go.
# This view could be materialized in a view and queried which would save lots repetitive aggregations
%%time
%%sql
SELECT dimDate.month,dimStore.country,sum(sales_amount) as revenue
FROM factSales
JOIN dimDate     on (dimDate.date_key         = factSales.date_key)
JOIN dimStore on (dimStore.store_key = factSales.store_key)
GROUP by cube(dimDate.month,  dimStore.country);
# 那么如果不用GROUP by cube的话那么naive的做法是：（注意UNION ALL）
%%time
%%sql
SELECT  NULL as month, NULL as country, sum(sales_amount) as revenue
FROM factSales
    UNION all 
SELECT NULL, dimStore.country,sum(sales_amount) as revenue
FROM factSales
JOIN dimStore on (dimStore.store_key = factSales.store_key)
GROUP by  dimStore.country
    UNION all 
SELECT cast(dimDate.month as text) , NULL, sum(sales_amount) as revenue
FROM factSales
JOIN dimDate on (dimDate.date_key = factSales.date_key)
GROUP by dimDate.month
    UNION all
SELECT cast(dimDate.month as text),dimStore.country,sum(sales_amount) as revenue
FROM factSales
JOIN dimDate     on (dimDate.date_key         = factSales.date_key)
JOIN dimStore on (dimStore.store_key = factSales.store_key)
GROUP by (dimDate.month, dimStore.country)


## copy data from csv to table
DROP TABLE IF EXISTS customer_reviews_row;
CREATE TABLE customer_reviews_row
(
    customer_id TEXT,
    review_date DATE,
    review_rating INTEGER,
    review_votes INTEGER,
    review_helpful_votes INTEGER,
    product_id CHAR(10),
    product_title TEXT,
    product_sales_rank BIGINT,
    product_group TEXT,
    product_category TEXT,
    product_subcategory TEXT,
    similar_product_ids CHAR(10)[]
)
COPY customer_reviews_row FROM '/tmp/customer_reviews_1998.csv' WITH CSV;

## cstore_fdw usage, extension for columar database
#Load extension: 
CREATE EXTENSION cstore_fdw;
#Create server object : 
CREATE SERVER cstore_server FOREIGN DATA WRAPPER cstore_fdw;
#Create foreign table:
CREATE FOREIGN TABLE events (
 event_date date,
 …
) 
SERVER cstore_server OPTIONS(compression 'pglz');
#Load data:
COPY events FROM '/tmp/events-2015-09-09.csv' WITH CSV;


# datatype of postgre
Boolean: boolean, bool
Character: CHAR(n)， VARCHAR(n), TEXT
Numeric:
    Integer: SMALLINT: 2-byte, -32768-32767
             INT: 4-byte
             Serial: is the same as integer except that PostgreSQL will automatically generate and populate values into the SERIAL column. 
                        This is similar to AUTO_INCREMENT column in MySQL or AUTOINCREMENT column in SQLite.
    Floating-point number: float(n), is a floating-point number whose precision, at least, n, up to a maximum of 8 bytes.
                            realor float8is a 4-byte floating-point number.
                            numeric or numeric(p,s) is a real number with p digits with s number after the decimal point. The numeric(p,s) is the exact number.
TIME:
    DATE stores the dates only.
    TIME stores the time of day values.
    TIMESTAMP stores both date and time values.
    TIMESTAMPTZ is a timezone-aware timestamp data type. It is the abbreviation for timestamp with the time zone.
    INTERVAL stores periods of time.

# 2 Cassandra

### Use a python wrapper/ python driver called cassandra to run the Apache Cassandra queries. This library should be preinstalled but in the future to install this library you can run this command in a notebook to install locally: 
`! pip install cassandra-driver`<br>
More documentation can be found here:  https://datastax.github.io/python-driver/

In [8]:
import cassandra

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

# The replication map determines how many copies of the data are kept in a given data center. 
### SimpleStrategy
#'replication_factor' : N  
#Assign the same replication factor to the entire cluster. Use for evaluation and single data center test and development environments only.

### NetworkTopologyStrategy
#'datacenter_name' : N    
#Assign replication factors to each data center in a comma separated list. Use in production environments and multi-DC test and development environments. 
#Data center names must match the snitch DC name; refer to Snitches for more details.

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

# We are working with Apache Cassandra a NoSQL database. We can't model our data and create our table without more information.
query = "CREATE TABLE IF NOT EXISTS music_library "
query = query + "(year int, artist_name text, album_name text, PRIMARY KEY (year, artist_name))"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "INSERT INTO music_library (year, artist_name, album_name)"
query = query + " VALUES (%s, %s, %s)"

try:
    session.execute(query, (1970, "The Beatles", "Let it Be"))
except Exception as e:
    print(e)
    
try:
    session.execute(query, (1965, "The Beatles", "Rubber Soul"))
except Exception as e:
    print(e)
    
query = "select * from music_library WHERE YEAR=1970"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.year, row.album_name, row.artist_name)
    
query = "drop table music_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
# Close the session and cluster connection   
session.shutdown()
cluster.shutdown()
    

In [ ]:
# 
- `Sakila` is a sample database created by `MySql` [Link](https://dev.mysql.com/doc/sakila/en/sakila-structure.html)
- The postgresql version of it is called `Pagila` [Link](https://github.com/devrimgunduz/pagila)